In [10]:
import pandas as pd
import numpy as np
import pickle

seed = 42
np.random.seed(seed)
lig_num = 1000
sample_size = 200
estimators = 100

In [11]:
with open("../models/tfidf.pickle", "rb") as fp:
    tfidf = pickle.load(fp)

In [12]:
df_sequences = pd.read_csv("../data/sequences.csv", index_col=0)
df_binding = pd.read_csv("../data/lig2seq.csv")

lig_id_vals = list(np.random.choice(df_binding["lig"].unique(), size=lig_num))
df_binding = df_binding.loc[df_binding["lig"].isin(lig_id_vals)]

In [13]:
from utils.baggingPU import BaggingClassifierPU
from sklearn.tree import DecisionTreeClassifier


models = {}
for i, lig_id in enumerate(lig_id_vals):
    print(i, len(lig_id_vals), len(models))
    seq_id = list(df_binding.loc[df_binding["lig"]==lig_id, "seq"])
    if len(seq_id) < sample_size//2 and len(seq_id) > 5:
        df_seq_sub_neg = df_sequences.loc[~df_sequences.index.isin(seq_id)].sample(n=sample_size-len(seq_id), random_state=seed+i)
        df_seq_sub_neg.loc[:,"bind"] = 0
        df_seq_sub_neg.loc[:,"bind_t"] = 0
        df_seq_sub_pos = df_sequences.loc[df_sequences.index.isin(seq_id)].sample(frac=0.75, random_state=seed+i)
        df_seq_sub_pos.loc[:,"bind"] = 1
        df_seq_sub_pos.loc[:,"bind_t"] = 1
        df_seq_sub_pos_hid_0 = df_sequences.loc[df_sequences.index.isin(seq_id)]
        df_seq_sub_pos_hid = df_seq_sub_pos_hid_0.loc[~df_seq_sub_pos_hid_0.index.isin(df_seq_sub_pos.index)]
        df_seq_sub_pos_hid.loc[:,"bind"] = 0
        df_seq_sub_pos_hid.loc[:,"bind_t"] = 1
        
        df_fitter = pd.concat([df_seq_sub_neg, df_seq_sub_pos, df_seq_sub_pos_hid])
        df_fitter["sequence"] = df_fitter["sequence"].apply(lambda x: tfidf.transform([x]))
        
        
        X = np.array([df_fitter["sequence"].values[i].toarray()[0] for i in range(df_fitter["sequence"].values.shape[0])])
        y = df_fitter["bind"].values
        
        bc = BaggingClassifierPU(DecisionTreeClassifier(), n_estimators=estimators, n_jobs=-1, max_samples=sum(y))
        bc.fit(X, y)
        df_seq_pos_all = df_sequences.loc[df_sequences.index.isin(seq_id)]
        df_seq_pos_all["seq_vec"] = df_seq_pos_all["sequence"].apply(lambda x: tfidf.transform([x]))
        df_seq_sub_neg["seq_vec"] = df_seq_sub_neg["sequence"].apply(lambda x: tfidf.transform([x]))
        print(
            lig_id,
            len(seq_id),
            bc.predict(np.array([df_seq_pos_all["seq_vec"].values[i].toarray()[0] for i in range(df_seq_pos_all["seq_vec"].values.shape[0])])).sum()/df_seq_pos_all.shape[0],
            bc.predict(np.array([df_seq_sub_neg["seq_vec"].values[i].toarray()[0] for i in range(df_seq_sub_neg["seq_vec"].values.shape[0])])).sum()/df_seq_sub_neg.shape[0]
        )
        models[lig_id] = pickle.dumps(bc)

0 1000 0


C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\brit2\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

592851 10 1.0 0.042105263157894736
1 1000 1
2 1000 1
3 1000 1


C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\brit2\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

1279516 10 1.0 0.04736842105263158
4 1000 2


...

1377857 12 1.0 0.031914893617021274
996 1000 537


C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\brit2\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

897130 7 1.0 0.02072538860103627
997 1000 538
998 1000 538


C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\brit2\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

1395314 8 1.0 0.3177083333333333
999 1000 539


In [14]:
len(models)

539

In [15]:
with open("../models/PUC.pickle", "wb") as fp:
    pickle.dump(models, fp)